In [ ]:
import pandas as pd
import json

# 1. Load the CSV Data
try:
    df = pd.read_csv('final_2000_2011.csv')
    print("CSV loaded successfully.")
except FileNotFoundError:
    print("Error: 'final_2000_2011.csv' not found. Please ensure it is in the same directory.")
    exit()

# 2. Select only the columns we need
# We neglect the risk_score as requested
data_subset = df[['Province', 'Year', 'disease_outbreak']].copy()

# 3. Clean Province names for better matching
# Remove leading/trailing spaces
data_subset['Province'] = data_subset['Province'].astype(str).str.strip()

# Convert data to a JSON string to embed in HTML
json_data = data_subset.to_json(orient='records')

# 4. Create the HTML Template
html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Vietnam Outbreak Map</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://d3js.org/d3-geo-projection.v4.min.js"></script>
    <style>
        body {{
            font-family: 'Segoe UI', sans-serif;
            background-color: #f4f4f4;
            display: flex;
            flex-direction: column;
            align-items: center;
            padding: 20px;
        }}
        h1 {{ color: #333; }}
        .controls {{
            background: white;
            padding: 15px 30px;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            margin-bottom: 20px;
            display: flex;
            align-items: center;
            gap: 15px;
        }}
        .year-display {{
            font-size: 24px;
            font-weight: bold;
            color: #007bff;
            min-width: 60px;
        }}
        input[type=range] {{
            width: 300px;
            cursor: pointer;
        }}
        #map-container {{
            width: 800px;
            height: 800px;
            background-color: white;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
            border-radius: 8px;
            position: relative;
        }}
        .province {{
            stroke: #fff;
            stroke-width: 0.5px;
            transition: fill 0.3s ease;
            cursor: pointer;
        }}
        .province:hover {{
            stroke: #333;
            stroke-width: 1.5px;
            opacity: 0.8;
        }}
        .legend {{
            margin-top: 15px;
            display: flex;
            gap: 20px;
        }}
        .legend-item {{
            display: flex;
            align-items: center;
            gap: 5px;
        }}
        .box {{ width: 20px; height: 20px; border-radius: 4px; }}
        .red {{ background: #e74c3c; }}
        .blue {{ background: #3498db; }}
        .gray {{ background: #ccc; }}
        
        /* Tooltip */
        .tooltip {{
            position: absolute;
            background: rgba(0,0,0,0.8);
            color: white;
            padding: 5px 10px;
            border-radius: 4px;
            pointer-events: none;
            opacity: 0;
            font-size: 14px;
        }}
    </style>
</head>
<body>

    <h1>Disease Outbreak Map (2000-2011)</h1>

    <div class="controls">
        <label>Year:</label>
        <input type="range" id="yearSlider" min="2000" max="2011" step="1" value="2000">
        <span id="yearVal" class="year-display">2000</span>
    </div>

    <div class="legend">
        <div class="legend-item"><div class="box red"></div> Outbreak (1)</div>
        <div class="legend-item"><div class="box blue"></div> No Outbreak (0)</div>
        <div class="legend-item"><div class="box gray"></div> No Data</div>
    </div>

    <div id="map-container">
        <div id="tooltip" class="tooltip"></div>
    </div>

    <script>
        // --- EMBEDDED DATA ---
        const csvData = {json_data};

        // --- MAP SETUP ---
        const width = 800;
        const height = 800;
        const svg = d3.select("#map-container").append("svg")
            .attr("width", width)
            .attr("height", height);
        
        const tooltip = d3.select("#tooltip");

        // Helper to normalize names for comparison
        // (Removes accents, lowercases, handles 'city' etc)
        function normalize(str) {{
            if (!str) return "";
            str = str.toLowerCase();
            str = str.normalize("NFD").replace(/[\u0300-\u036f]/g, ""); // Remove accents
            str = str.replace(/đ/g, "d");
            // Remove common prefixes/suffixes
            str = str.replace(/^(tinh|thanh pho|tp\.|tp) /, "");
            str = str.replace(" city", ""); 
            return str.replace(/[^a-z0-9 ]/g, "").trim(); 
        }}

        d3.json("diaphantinh.json").then(function(geoData) {{

            // Projection
            const projection = d3.geoCylindricalStereographic()
                .fitSize([width, height], geoData);
            const path = d3.geoPath().projection(projection);

            // Draw Map
            const provinces = svg.selectAll("path")
                .data(geoData.features)
                .enter().append("path")
                .attr("d", path)
                .attr("class", "province")
                .attr("fill", "#ccc") // Default
                .on("mouseover", function(event, d) {{
                    const currentYear = document.getElementById("yearSlider").value;
                    const status = getStatus(d.properties.ten_tinh, currentYear);
                    let label = "No Data";
                    if(status === 1) label = "Outbreak";
                    if(status === 0) label = "No Outbreak";

                    tooltip.style("opacity", 1)
                           .html(`<strong>${{d.properties.ten_tinh}}</strong><br>${{label}}`)
                           .style("left", (event.pageX + 10) + "px")
                           .style("top", (event.pageY - 20) + "px");
                    
                    d3.select(this).raise(); // Bring to front
                }})
                .on("mousemove", function(event){{
                     tooltip.style("left", (event.pageX + 10) + "px")
                            .style("top", (event.pageY - 20) + "px");
                }})
                .on("mouseout", function() {{
                    tooltip.style("opacity", 0);
                }});

            // Logic to find data
            function getStatus(geoName, year) {{
                const normGeo = normalize(geoName);
                
                // Find matching row
                const row = csvData.find(d => {{
                    return d.Year == year && normalize(d.Province) === normGeo;
                }});

                return row ? row.disease_outbreak : null;
            }}

            // Update function
            function updateMap(year) {{
                provinces.transition().duration(200)
                    .attr("fill", d => {{
                        const status = getStatus(d.properties.ten_tinh, year);
                        if (status === 1) return "#e74c3c"; // Red
                        if (status === 0) return "#3498db"; // Blue
                        return "#ccc"; // Gray/No Data
                    }});
            }}

            // Initialize
            updateMap(2000);

            // Slider Event
            const slider = document.getElementById("yearSlider");
            const valDisplay = document.getElementById("yearVal");
            
            slider.addEventListener("input", function() {{
                const year = this.value;
                valDisplay.textContent = year;
                updateMap(year);
            }});

        }}).catch(err => {{
            console.error(err);
            document.body.innerHTML += `<p style='color:red; text-align:center'>Error loading diaphantinh.json</p>`;
        }});

    </script>
</body>
</html>
"""

# 5. Write to File
with open("vietnam_outbreak_map.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("Successfully created 'vietnam_outbreak_map.html'")

CSV loaded successfully.
Successfully created 'vietnam_outbreak_map.html'


In [3]:
import pandas as pd
import json

def convert_csv_to_js_const(input_csv, output_txt, var_name="csvData"):
    # 1. Read the CSV file
    # Ensure your CSV has headers: Province, Year, disease_outbreak
    df = pd.read_csv(input_csv)
    
    # 2. Convert DataFrame to a list of dictionaries
    # 'records' format: [{column -> value}, ... , {column -> value}]
    data_dict = df.to_dict(orient='records')
    
    # 3. Format as a JavaScript constant
    # json.dumps ensures strings are quoted correctly and numbers are not
    js_const_string = f"const {var_name} = {json.dumps(data_dict, ensure_ascii=False)};"
    
    # 4. Save to a .txt file
    with open(output_txt, 'w', encoding='utf-8') as f:
        f.write(js_const_string)
    
    print(f"Successfully converted {input_csv} to {output_txt}")

if __name__ == "__main__":
    # Change 'input.csv' to your actual filename
    convert_csv_to_js_const(r"C:\Users\ADMIN\Downloads\vietnam_2000_2024_no_var.csv", 'javascript_const.txt')

Successfully converted C:\Users\ADMIN\Downloads\vietnam_2000_2024_no_var.csv to javascript_const.txt
